## 1. Import Libraries

In [92]:
import pickle

from skimage.transform import resize
import numpy as np
import cv2

## 2. Parameters

In [93]:
mask_path = "./data/mask_1920_1080.png"
video_path = "./data/parking_1920_1080.mp4"
model_path = './model/svc_best_est.p'

blue = (255, 0, 0)
green = (0, 255, 0)
red = (0, 0, 255)
line_wt = 2

ret = True
frames_per_clf = 30
frame_num = 0

## 3. Define Functions

In [94]:
def get_parking_spots_boxes(cc):
    (total_con_comp, comp_label_ids, values, centroid) = cc

    # Empty list 'slots' to store the bounding boxes of the parking spots.
    slots = []
    coef = 1
    for i in range(1, total_con_comp):

        # Now extract the coordinate points
        x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
        y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
        w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
        h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

        slots.append([x1, y1, w, h])

    return slots

def get_coor(xywh):
    x1, y1, w, h = xywh
    top_left = (x1, y1)
    bottom_right = (x1 + w, y1 + h)
    
    return x1, y1, w, h, top_left, bottom_right

def is_empty(spot_bgr, model_path):

    flat_data = []

    img_resized = resize(spot_bgr, (15, 15, 3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)

    MODEL = pickle.load(open(model_path, "rb"))
    y_output = MODEL.predict(flat_data)

    if y_output == 0:
        return True
    else:
        return False
    
def calc_diff(img1, img2):
    return np.abs(np.mean(img1) - np.mean(img2))

def display_text(frame, spots_status):
    text = f"Available Lots: {str(sum(spots_status))} / {str(len(spots_status))}"
    white = (255, 255, 255)
    cv2.putText(frame, text, (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, white, 2)

## 4. Masking

In [95]:
# read grayscale image of mask
mask = cv2.imread(mask_path, 0)
cap = cv2.VideoCapture(video_path)

mask[0] # First row of pixels

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [96]:
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
parking_spots_xywh = get_parking_spots_boxes(connected_components)

## 5. Start Test Video

In [ ]:
spots_status = [None for i in parking_spots_xywh]
diffs = [None for i in parking_spots_xywh]
prev_frame = None

# Continues as long as ret is True, indicating that a frame was successfully read.
while ret:
    # Retrieves next frame from video
    ret, frame = cap.read()
    
    # For every 30 frames )if frame_num is divisible by 30)
    if frame_num % frames_per_clf == 0:
        if prev_frame is not None:
            for index, xywh in enumerate(parking_spots_xywh):
                x1, y1, w, h, _, _ = get_coor(xywh)
                cropped_spot = frame[y1:y1+h, x1:x1+w, :]
                diffs[index] = calc_diff(cropped_spot, prev_frame[y1:y1+h, x1:x1+w, :])
                
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.2]
        else:
            arr_ = range(len(parking_spots_xywh))
            
        for index in arr_:
            xywh = parking_spots_xywh[index]
            # Get all coordinates
            x1, y1, w, h, _, _ = get_coor(xywh)
            # Define coordinates for parking spot only
            cropped_spot = frame[y1:y1+h, x1:x1+w, :]
            
            # Use model to predict if cropped_spot is empty or not
            is_it_empty = is_empty(cropped_spot, model_path)
            # Store prediction as True or False
            spots_status[index] = is_it_empty

    if frame_num % frames_per_clf == 0:
        prev_frame = frame.copy()

    for index, xywh in enumerate(parking_spots_xywh):
        _, _, _, _, top_left, bottom_right = get_coor(xywh)
        
        is_it_empty = spots_status[index]
            
        if is_it_empty is True:
            frame = cv2.rectangle(frame,top_left,bottom_right,green,line_wt)
        else:
            frame = cv2.rectangle(frame,top_left,bottom_right,red,line_wt)
    
    display_text(frame, spots_status)
    
    # Display the current frame, in a window named 'frame'
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    
    frame_num += 1
    
cap.release()
cv2.destroyAllWindows()